# Example of using ioh-explainer

First we need to define the configuration space of an algorithm. We take a standard PSO algorithm as example.

In [1]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

from ioh_xplainer import explainer
#import pyswarms as ps
from modde import ModularDE, Parameters
import numpy as np
from ConfigSpace import ConfigurationSpace
from ConfigSpace.util import generate_grid
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import autokeras as ak

cs = ConfigurationSpace({
    "F": (0.05, 2.0),              # Uniform float
    "CR" : (0.05, 1.0),            # Uniform float
    "lambda_": (1, 20)             # Uniform int
})

steps_dict = {
    "F": 20, 
    "CR" : 20,
    "lambda_": 10
}

def run_de(func, config, budget, dim, *args, **kwargs):
    item = {'F': np.array([float(config.get('F'))]), 'CR':np.array([float(config.get('CR'))]),  'lambda_' : int(config.get('lambda_'))*dim }
    item['budget'] = int(budget)
    c = ModularDE(func, **item)
    try:
        c.run()
        return []
    except Exception as e:
        print(f"Found target {func.state.current_best.y} target, but exception ({e}), so run failed")
        return []

de_explainer = explainer(run_de, 
                 cs , 
                 dims = [5,10,20],#,10,40],#, 10, 20, 40 
                 fids = [1,5,7,13,18,20,23], #,5
                 iids = 5, #20 
                 reps = 5, 
                 sampling_method = "grid",  #or random
                 grid_steps_dict = steps_dict,
                 sample_size = None,  #only used with random method
                 budget = 10000, #10000
                 seed = 1,
                 verbose = True)

Using TensorFlow backend


2023-08-03 12:18:11.292004: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-03 12:18:12.516645: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
if False:
    de_explainer.run(paralell=True)
    #de_explainer.save_results("de_results.pkl")

In [3]:
de_explainer.load_results("de_results.pkl")
de_explainer._create_grid()

x = de_explainer.df[(de_explainer.df['fid'] == 1) & (de_explainer.df['dim'] == 5)][["F","CR","lambda_"]].to_numpy()

#y = de_explainer.df[(de_explainer.df['fid'] == 1) & (de_explainer.df['dim'] == 5)]["auc"].to_numpy()
#np.savetxt("sobol/x.csv", x)
#np.savetxt("sobol/y.csv", y)

#de_explainer.plot(partial_dependence=False, best_config=False)


Evaluating 4000 configurations.


In [4]:
print(de_explainer.performance_stats())

             d=5                                               \
        Function           single-best               avg-best   
0         Sphere  7228.42 (9717088.00)  7421.97 (10399794.84)   
1    LinearSlope   3560.97 (115063.22)    3560.97 (115063.22)   
2  StepEllipsoid    3528.80 (38734.02)   3915.63 (1676877.40)   
3     SharpRidge    3165.82 (16321.60)     3165.82 (16321.60)   
4  Schaffers1000   3904.31 (255960.14)   4817.97 (5387299.22)   
5       Schwefel      2229.93 (885.94)   3178.48 (4239646.07)   
6       Katsuura  6388.20 (9110326.67)   6503.68 (8604636.85)   

                                  d=10                        \
                    avg       Function           single-best   
0  7616.98 (8870262.14)         Sphere  5514.11 (5407340.08)   
1  8815.62 (4051237.21)    LinearSlope    3437.49 (62995.67)   
2  4950.28 (3810684.64)  StepEllipsoid     3162.96 (1950.20)   
3  4827.10 (4831908.71)     SharpRidge     2951.72 (2383.44)   
4  5144.79 (4626474.37)  Schaf

In [5]:
df = de_explainer.behaviour_stats()
df.style.bar(cmap='viridis')

#do it for f1 and f5
df = de_explainer.behaviour_stats(fids=[1,5])
df.style.bar(cmap='viridis')